In [ ]:
import os
from pathlib import Path 
import allego_file_reader as afr
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import sys
sys.path.append('/Users/ebennett/Library/CloudStorage/Dropbox-EMI/Evelyn Bennett/project_electrodes/electrode_data/electrode-analysis/Code')
import xdat_loader
import figure_fxns



In [ ]:
#reloads if nessesary
import importlib
importlib.reload(xdat_loader)
importlib.reload(figure_fxns)

In [ ]:
# directory to the folder containing the data files
# change to "filname" after done testing
target_dir = "/Users/ebennett/Library/CloudStorage/Dropbox-EMI/Evelyn Bennett/project_electrodes/electrode_data/electrode-analysis/Data/20260130_Liver8_biopsy4" # data directory

In [ ]:
# save path for files
figures_dir = "/Users/ebennett/Library/CloudStorage/Dropbox-EMI/Evelyn Bennett/project_electrodes/electrode_data/electrode-analysis/Figures/20260130_Liver8_biopsy2"

In [ ]:
# check the file indices
xdat_loader.print_all_xdat_names(target_dir)
# creats an array of the indices to feed to the dataframe builder fxn
index = xdat_loader.get_xdat_indices(target_dir)

In [ ]:
# build the full dataframe
"""
--------- Structure Reminders ---------
all channels from one dataset
df['crc-after-kcl_0']

one channel across all datasets
df.xs(5, level='channel', axis=1)

one dataset, one channel
df[('crc-after-kcl_0', 5)]
"""
master_df = xdat_loader.build_dataframe(target_dir, index)



In [ ]:
#assign color seperately
# assign a color to each file so colors are consistent between graphs
file_colors =xdat_loader.assign_colors(master_df)

In [ ]:
# this adds a set of display names to the dataset for easier plotting
# display names are manuelly set
display_names = [
    "Spont Recording",
    "PBSdrop Recording",
    "PBS-Glu Sequence Drop Recording",
    "PBS Recording",
    "Glu Recording",
    "Glu Sequence Drop Recording",
    "KCl Recording"
]

dataset_index = np.arange(0,7)

display_dict = xdat_loader.add_display_names(master_df, dataset_index, display_names)

In [ ]:
print(master_df.columns)
print(master_df.index[:10])

In [ ]:
import figure_fxns
# generate the full signal of 1 channel
channel_id = 3
name = 'KCl_recording_0'
t_start = 0
t_end=50
figure_fxns.plot_channel(
    master_df, 
    channel_id, 
    name, 
    t_start, 
    t_end,
    #ylim=((-8000, 9000)),
    figures_dir=figures_dir 
)

In [ ]:
import figure_fxns
# generate the full signal of 1 channel
channel_id = 3
name = 'KCl_recording_0'
t_start = 33
t_end=33.6
figure_fxns.plot_channel(
    master_df, 
    channel_id, 
    name, 
    t_start, 
    t_end,
    #ylim=((-8000, 9000)),
    figures_dir=figures_dir 
)

In [ ]:
figure_fxns.raster_plot(
    master_df,
    channel_id,
    t_start,
    t_end,
    offset=1.0,
    figures_dir=figures_dir
)

In [ ]:

t0_1 = 36.1  # seconds
t1_1 = 36.9

t0_2 = 22.7
t1_2 = 23.5

dataset_names = ['crc-after-kcl_0', 'crc-after-kcl-dose2_0']

time_slices = {
    'crc-after-kcl_0': (t0_1, t1_1),       # seconds
    'crc-after-kcl-dose2_0': (t0_2, t1_2),
}

# Create list of pre-sliced dfs
# in this case you cant just feed a satrt and end time bc the spike segments are at diffferent time points
dfs_sliced = []
for dataset_name, (t_start, t_end) in time_slices.items():
    # Slice the master_df by index using the time column
    df_slice = master_df.loc[t_start:t_end, (dataset_name, channel_id)]
    
    # Convert back to a DataFrame with the same MultiIndex structure
    df_slice = pd.DataFrame(df_slice)
    df_slice.columns = pd.MultiIndex.from_tuples([(dataset_name, channel_id)], names=['dataset', 'channel'])
    
    dfs_sliced.append(df_slice)

figure_fxns.spike_bar_graphs_all(
    dfs_sliced,      # use the sliced datasets
    channel_id,
    master_df,       # still used for colors and display names
    figures_dir
)